In [1]:
import csv
from bs4 import BeautifulSoup
import requests

In [2]:
base_url = "https://www.olx.uz"
url = 'https://www.olx.uz/oz/nedvizhimost/kvartiry/prodazha/tashkent/?currency=UYE'

In [3]:
def get_href(url_): # get hrefs of all houses from url  one by one
    page = requests.get(url_)
    soup = BeautifulSoup(page.content, 'html.parser')
    houses1 =[]


    # Find all divs with class 'something'
    tags = soup.find_all("a", class_="css-1tqlkj0")
    for a_tag in tags:
      houses1.append(base_url + a_tag["href"])
    return houses1

In [4]:
#get first page
houses = get_href(url)
#get all remaining pages
for i in range(2,25):
    houses.extend(get_href(url+'&page='+str(i)))
# remove duplicates
houses = list(dict.fromkeys(houses))
print(len(houses))

1235


In [5]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_house_data(house_url, i=None):
    try:
        page = requests.get(house_url, timeout=10)
        soup = BeautifulSoup(page.content, 'html.parser')


        tags = soup.find("div", class_="css-41yf00")
        if tags is None:
            return None  # Skip this URL
        # 1. scrape main house data
        p_tags = tags.find_all("p")
        tag_content = {}
        for tag in p_tags:
            l = tag.text.split(':')
            if len(l) > 1:
                tag_content[l[0].strip()] = l[1].strip()
        # 2.  find district location
        loc_tag = soup.find("p", class_="css-7wnksb")
        district = loc_tag.text.split(',')
        if len(district) > 1:
                tag_content['Tuman'] = district[1].split()[0].strip()
        # 3. Get price
        price_tag = soup.find("h3", class_="css-fqcbii")
        price = price_tag.text.split()
        tag_content['Narx'] = price[0]
        # end
        if i is not None:
            print(f"[{i}] Fetched: {house_url}")
        return tag_content

    except Exception as e:
        print(f"Error fetching {house_url}: {e}")
        return None

def get_data_parallel(house_url_list, max_workers=12):
    house_content = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_house_data, url, i): url for i, url in enumerate(house_url_list)}
        for future in as_completed(futures):
            result = future.result()
            if result:
                house_content.append(result)

    return house_content

data = get_data_parallel(houses, max_workers=12)


In [6]:
features = ['Turarjoy turi','Xonalar soni','Umumiy maydon','Qavati', 'Uy qavatliligi','Mebelli','Tuman','Taʼmiri','Narx']

In [7]:
import csv
# save data to csv file
with open('../housing_data.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=features)
    writer.writeheader()
    for row in data:
        writer.writerow({k: row.get(k, '') for k in features})


data analysis

In [8]:
import csv
# read data to csv file
with open('../housing_data.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f) # it gets headers and makes dictionaries  for every row
    data = [row for row in reader]
data

[]

In [9]:
# distinct districts
districts = set ([data[i]['Tuman'] for i in range(len(data))])
dt ={}
for item in districts:
    dt[item]= 0
for i in range(len(data)):
    dt[data[i]['Tuman']] +=1
print(dt)
print(dt.keys())

{}
dict_keys([])
